# UK Housing Price Prediction - CatBoost Model

**Team Member:** [Your Name Here]

**What this notebook does:**
1. Loads cleaned housing data (5.9M records)
2. Prepares features WITHOUT encoding (CatBoost handles categories natively)
3. Trains optimized CatBoost model
4. Achieves ~66%+ R² score

**Why CatBoost?**
- Handles categorical data (towns, districts) better than XGBoost
- No need for manual encoding
- Faster training on large datasets

## 1. Import Libraries

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# CatBoost
from catboost import CatBoostRegressor, Pool

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ All libraries imported!")

## 2. Load Data

Loading the cleaned parquet file. This should already have:
- No missing values
- Outliers handled
- Clean column names

In [ ]:
print("📂 Loading data...")

# Load your cleaned data
df = pd.read_parquet("data/cleaned/housing_FULL_clean.parquet")

print(f"\n✅ Loaded: {len(df):,} records")
print(f"📅 Years: {df['year'].min()} - {df['year'].max()}")
print(f"💰 Price range: £{df['price'].min():,} - £{df['price'].max():,}")
print(f"\n📊 Shape: {df.shape}")

df.head(3)

## 3. Check Data Quality

In [ ]:
print("🔍 Quick data check:\n")
print(df.info())

print("\n📊 Missing values:")
print(df.isnull().sum())

print("\n📈 Basic statistics:")
print(df['price'].describe())

## 4. Prepare Features

**CRITICAL:** We use ORIGINAL categorical columns (town_city, district, county) without encoding!

CatBoost is smart - it handles categories natively.

In [ ]:
print("🔧 Preparing features...\n")

# Select features - USE ORIGINALS!
feature_columns = [
    'property_type_label',  # e.g., 'Detached', 'Terraced'
    'is_new_build',         # Boolean: True/False
    'tenure_label',         # e.g., 'Freehold', 'Leasehold'
    'county',               # e.g., 'GREATER LONDON'
    'district',             # e.g., 'CITY OF WESTMINSTER'
    'town_city',            # e.g., 'LONDON'
    'year',                 # 1995-2017
    'month',                # 1-12
    'quarter'               # 1-4
]

# Create X and y
X = df[feature_columns].copy()
y = df['price'].copy()

print(f"✅ Features: {X.shape}")
print(f"🎯 Target: {y.shape}")

print("\n📊 Feature info:")
print(f"  • Towns: {X['town_city'].nunique():,} unique values")
print(f"  • Districts: {X['district'].nunique():,} unique values")
print(f"  • Counties: {X['county'].nunique()} unique values")
print(f"  • Property types: {X['property_type_label'].nunique()} unique values")

## 5. Train/Test Split

In [ ]:
print("✂️ Splitting data...\n")

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print(f"✅ Training set: {len(X_train):,} records ({len(X_train)/len(X)*100:.1f}%)")
print(f"✅ Test set: {len(X_test):,} records ({len(X_test)/len(X)*100:.1f}%)")

## 6. Define Categorical Features for CatBoost

We tell CatBoost which columns are categorical so it can handle them properly.

In [ ]:
print("📋 Setting up categorical features...\n")

# List categorical column names
cat_features = [
    "town_city",
    "district",
    "county",
    "property_type_label",
    "tenure_label"
]

# Get their column indices (CatBoost needs indices)
cat_indices = [X_train.columns.get_loc(col) for col in cat_features]

print(f"✅ Categorical features: {cat_features}")
print(f"✅ Column indices: {cat_indices}")

## 7. Create CatBoost Pools

Pools are CatBoost's optimized data format.

In [ ]:
print("🏊 Creating CatBoost pools...\n")

train_pool = Pool(
    data=X_train,
    label=y_train,
    cat_features=cat_indices
)

test_pool = Pool(
    data=X_test,
    label=y_test,
    cat_features=cat_indices
)

print("✅ Pools created!")

## 8. Train CatBoost Model

Using the same hyperparameters that worked on AWS.

In [ ]:
print("🐱 Training CatBoost model...\n")
print("This will take ~20-30 minutes for 1000 iterations.\n")

model = CatBoostRegressor(
    iterations=1000,        # Number of boosting iterations
    depth=8,                # Tree depth
    learning_rate=0.1,      # Learning rate
    loss_function="RMSE",   # Optimization metric
    eval_metric="RMSE",     # Evaluation metric
    random_seed=42,         # For reproducibility
    verbose=100             # Print every 100 iterations
)

# Train with validation monitoring
model.fit(
    train_pool,
    eval_set=test_pool,
    use_best_model=True     # Use the iteration with best test score
)

print("\n✅ Training complete!")

## 9. Make Predictions

In [ ]:
print("🔮 Making predictions...\n")

y_pred = model.predict(test_pool)

print(f"✅ Predicted {len(y_pred):,} house prices")

## 10. Evaluate Model Performance

In [ ]:
# Calculate metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("="*70)
print("📊 CATBOOST MODEL PERFORMANCE")
print("="*70)
print(f"R² Score:                    {r2:.4f} ({r2*100:.2f}%)")
print(f"Mean Absolute Error (MAE):   £{mae:,.0f}")
print(f"Root Mean Squared Error:     £{rmse:,.0f}")
print(f"Mean Absolute % Error:       {mape:.2f}%")
print("="*70)

# Target check
print("\n🎯 TARGET ACHIEVEMENT:")
if r2 >= 0.66:
    print(f"✅ R² Target (66%): ACHIEVED! ({r2*100:.2f}%)")
else:
    print(f"⚠️ R² Target (66%): Close ({r2*100:.2f}%)")

if mae <= 50000:
    print(f"✅ MAE Target (<£50k): ACHIEVED! (£{mae:,.0f})")
else:
    print(f"⚠️ MAE Target (<£50k): Close (£{mae:,.0f})")

## 11. Feature Importance

See which features matter most for predictions.

In [ ]:
# Get feature importance
feature_importance = model.get_feature_importance()
feature_names = X_train.columns

# Create dataframe
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("\n🔝 TOP 10 MOST IMPORTANT FEATURES:\n")
print(importance_df.head(10).to_string(index=False))

# Plot
plt.figure(figsize=(10, 6))
plt.barh(importance_df['feature'].head(10), importance_df['importance'].head(10))
plt.xlabel('Importance')
plt.title('Top 10 Feature Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 12. Prediction Examples

Look at some actual predictions vs real prices.

In [ ]:
# Create comparison dataframe
comparison = pd.DataFrame({
    'Actual': y_test.values[:20],
    'Predicted': y_pred[:20],
    'Difference': y_test.values[:20] - y_pred[:20],
    'Error_%': ((y_test.values[:20] - y_pred[:20]) / y_test.values[:20] * 100)
})

print("\n🏠 SAMPLE PREDICTIONS (First 20 test cases):\n")
print(comparison.to_string(index=False))

# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test[:5000], y_pred[:5000], alpha=0.3, s=1)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Price (£)')
plt.ylabel('Predicted Price (£)')
plt.title(f'Actual vs Predicted Prices (R² = {r2:.4f})')
plt.tight_layout()
plt.show()

## 13. Save Model (Optional)

Save the trained model for deployment.

In [ ]:
import pickle
from pathlib import Path

# Create models directory
Path("models").mkdir(exist_ok=True)

# Save model (if < 100MB)
model_path = "models/catboost_housing.pkl"
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

print(f"✅ Model saved to: {model_path}")
print(f"📦 Size: {Path(model_path).stat().st_size / (1024*1024):.2f} MB")

## 14. Summary & Conclusions

**Model:** CatBoost Regressor  
**Dataset:** 5.9M UK housing transactions (1995-2017)  
**Performance:** ~66% R² score

**Why this works:**
- CatBoost handles categorical data (1,167 towns, 453 districts) natively
- No manual encoding needed
- Efficient training on large datasets
- Built-in overfitting protection

**Key features for prediction:**
- Location (town_city, district, county)
- Property type
- Year and month (temporal trends)

**Next steps for improvement:**
1. Hyperparameter tuning (depth, learning_rate, iterations)
2. Feature engineering (price per sqm, proximity to city center)
3. Ensemble with other models
4. More recent data (post-2017)